In [6]:
import cv2
import numpy as np
import easyocr
import torch
import re
from collections import deque, Counter
from difflib import SequenceMatcher

print(torch.__version__)
print(torch.version.cuda)
print(torch.backends.cudnn.version())
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"Number of GPUs: {torch.cuda.device_count()}")
print(f"cuDNN enabled: {torch.backends.cudnn.enabled}")

# Carregar o modelo EasyOCR uma vez fora do loop
leitor = easyocr.Reader(['pt', 'en'], gpu=True)

# Função para melhorar a qualidade da imagem antes do OCR
def preprocess_image(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    contrast_enhanced = cv2.equalizeHist(gray)
    
    kernel = np.array([[0, -1, 0], 
                       [-1, 5, -1],
                       [0, -1, 0]])
    sharpened = cv2.filter2D(contrast_enhanced, -1, kernel)
    
    smoothed = cv2.GaussianBlur(sharpened, (3, 3), 0)
    
    return smoothed

# Função para verificar se o texto tem 7 caracteres
def verificar_formato_placa(texto):
    return len(texto) == 7

# Função para calcular similaridade entre duas strings
def calcular_similaridade(str1, str2):
    return SequenceMatcher(None, str1, str2).ratio()

# Função para realizar OCR diretamente no frame
def detectar_caracteres(img):
    img_preprocessada = preprocess_image(img)
    
    allowlist = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'
    resultado = leitor.readtext(img_preprocessada, allowlist=allowlist)
    
    texto_detectado = " ".join([res[1] for res in resultado])
    
    if verificar_formato_placa(texto_detectado):
        return texto_detectado
    else:
        return None

# Função para contar caracteres e usar sistema de votação por caractere
def atualizar_votacao_caracteres(placa_atual, nova_placa, votos_caracteres):
    for i, caractere in enumerate(nova_placa):
        if i < len(placa_atual):
            # Adicionar o caractere ao contador de votos na posição i
            votos_caracteres[i].update([caractere])

# Função para formar a placa mais provável com base nos votos de cada caractere
def formar_placa_por_votacao(votos_caracteres):
    return "".join([votos.most_common(1)[0][0] for votos in votos_caracteres])

# Função para detectar se uma nova placa deve ser considerada
def detectar_nova_placa(placa_atual, nova_placa):
    # Contar quantos caracteres são diferentes entre as placas
    diferencas = sum([1 for a, b in zip(placa_atual, nova_placa) if a != b])
    return diferencas >= 4  # Se 4 ou mais caracteres forem diferentes, é uma nova placa

# Função para processar o vídeo frame a frame e salvar o resultado
def processar_video(video_path, output_path, skip_frames=5):  # Novo argumento para pular frames
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Erro ao carregar o vídeo.")
        return

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    placa_atual = None  # Placa atual inferida
    votos_caracteres = []  # Lista de contadores de votos por posição de caractere

    frame_count = 0  # Contador de frames
    while cap.isOpened():
        ret, frame = cap.read()
        if ret:
            # Processar apenas 1 a cada 'skip_frames'
            if frame_count % skip_frames == 0:  
                texto_detectado = detectar_caracteres(frame)
                if texto_detectado is not None:
                    if placa_atual is None:
                        # Primeira placa detectada
                        placa_atual = texto_detectado
                        votos_caracteres = [Counter() for _ in range(len(placa_atual))]
                        print(f"Nova placa detectada: {placa_atual}")
                    else:
                        if detectar_nova_placa(placa_atual, texto_detectado):
                            # Considerar uma nova placa
                            placa_atual = texto_detectado
                            votos_caracteres = [Counter() for _ in range(len(placa_atual))]
                            print(f"Nova placa detectada: {placa_atual}")
                        else:
                            # Atualizar os votos de cada caractere
                            atualizar_votacao_caracteres(placa_atual, texto_detectado, votos_caracteres)
                            # Formar a nova placa inferida com base nos votos
                            placa_inferida = formar_placa_por_votacao(votos_caracteres)
                            if placa_inferida != placa_atual:
                                placa_atual = placa_inferida
                                print(f"Placa inferida: {placa_atual}")

            # Exibir a placa atual no vídeo em todos os frames, se houver uma placa detectada
            if placa_atual:
                cv2.putText(frame, f"Placa: {placa_atual}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            # Gravar o frame no arquivo de saída
            out.write(frame)
            frame_count += 1
        else:
            break

    cap.release()
    out.release()

# Chamar a função com o caminho do vídeo de entrada e de saída
video_path = 'organic-data/video_teste_2.mp4'
output_path = 'resultado_textscan_vote.mp4'
processar_video(video_path, output_path, skip_frames=2) 


2.4.1+cu118
11.8
90100
CUDA available: True
Number of GPUs: 1
cuDNN enabled: True
Nova placa detectada: MVJ0317
Nova placa detectada: PKH8G22
Placa inferida: KKH8G22
Placa inferida: NKH8G22
Nova placa detectada: OFG8016
Placa inferida: 0FG8016
Placa inferida: OFG8016
Placa inferida: 0FG8016
Placa inferida: OFG8016
Nova placa detectada: 5 J8C02
Placa inferida: S 18C02
Nova placa detectada: 0HJ8CO2
Placa inferida: 0HJ8C02
Nova placa detectada: 07B0AD2
Placa inferida: 0JBTAD2
Nova placa detectada: 00JBMD2
Nova placa detectada: 0JBUAD2
Placa inferida: OJBGAD2
Placa inferida: OJBDAD2
Placa inferida: OJBGAD2
Placa inferida: OJBDAD2
Placa inferida: OJBGAD2
Placa inferida: 0JBGAD2
Placa inferida: OJBGAD2
Nova placa detectada: 00B6A22
Placa inferida: 0IBGAD2
Placa inferida: 0IBOAD2
Nova placa detectada: OGB4371
Placa inferida: 0GB4371
